In [ ]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from CV.nn.conv.MiniVGGNet import MiniVGGNet
from keras.optimizers import SGD
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import argparse

# construct the argument parse and parse the argument in commond line
ap = argparse.ArgumentParser()
ap.add_argument("-o","--output",
                required = True, help="path to the ouput loos/accuracy plot")
args = vars(ap.parse_args)

print("[INFO] loading CIFAR-10 data...")
((train_X,train_y),(test_X,test_y)) = cifar10.load_data()
train_X = train_X.astype("float")/255.0
test_X = test_X.astype("float")/255.0

lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y = lb.fit_transform(test_y)

# initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer",
              "dog", "frog", "horse", "ship", "truck"]

print("[INFO] compiling model...")
opt = SGD(lr=0.01, decay=0.01/40, momentum=0.9,nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,metrics=["accuracy"])
# train
H = model.fit(train_X, train_y, validation_data=(test_X,test_y),
              epochs = 40, batch_size=64, verbose=1)

# evaluate
print("[INFO] evaluating network...")
predictions = model.predict(test_X, batch_size=64)
print(classification_report(test_y.argmax(axis=1),predictions.argmax(axis=1),
      target_names=labelNames))

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 40), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 40), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 40), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 40), H.history["val_acc"], label="val_acc")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

Using TensorFlow backend.
